# AAI614: Data Science & its Applications

*Notebook 4.1: Graph Analytics with cuGraph and TIGER*

<a href="https://colab.research.google.com/github/harmanani/AAI614/blob/main/Week%204/Notebook4.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




The study of network robustness is critical to the understanding of complex interconnected systems. For example, consider an example of a power grid network that is susceptible to both natural failures and targeted attacks. A natural failure occurs when a single power substation fails due to erosion of parts or natural disasters. However, when one substation fails, additional load is routed to alternative substations, potentially causing a series of cascading failures. Not all failures originate from natural causes, some come from targeted attacks, such as enemy states hacking into the grid to sabotage key equipment to maximally damage the operations of the electrical grid. A natural counterpart to network robustness is vulnerability, defined as measure of a network’s susceptibility to the dissemination of entities across the network, such as how quickly a virus spreads across a computer network.

In this lab, we show how to use [cuGraph](https://github.com/rapidsai/cugraph) and [TIGER](https://github.com/safreita1/TIGER) to conduct state-of-the-art GPU accelerated graph vulnerability and robustness analysis. Specifically, we will look at how to:

- *Quantify network vulnerability and robustness* (**Part 1**),
- *Simulate network attacks and cascading failures on networks* (**Part 2**)
- *Regulate the dissemination of computer virues on a network* (**Part 3**)

Lab Source: **NVIDIA**

## Setup
Lets begin by installing the following 2 libraries:

1.   Graph vulnerability and robustness analysis library: [TIGER](https://github.com/safreita1/TIGER)
2.   GPU acceleration library: [CuPy](https://github.com/cupy/cupy)


In [1]:
pip install graph-tiger

In [2]:
import os
print("Current working directory:", os.getcwd())
print("Contents of current directory:")
for item in os.listdir('.'):
    print(f"  {item}")

Current working directory: /content
Contents of current directory:
  .config
  rapidsai-csp-utils
  sample_data


In [3]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.


In [4]:
import os
file_path = 'rapidsai-csp-utils/colab/update_modules.py'
if os.path.exists(file_path):
    print("File exists!")
else:
    print("File not found. Let's check the repository structure:")
    if os.path.exists('rapidsai-csp-utils'):
        print("Repository exists. Contents:")
        for root, dirs, files in os.walk('rapidsai-csp-utils'):
            for file in files:
                if file.endswith('.py'):
                    print(f"  {os.path.join(root, file)}")
    else:
        print("Repository not found.")

File not found. Let's check the repository structure:
Repository exists. Contents:
  rapidsai-csp-utils/colab/install_rapids.py
  rapidsai-csp-utils/colab/env-check.py
  rapidsai-csp-utils/colab/pip-install.py


In [5]:
# Check what install_rapids.py does
with open('rapidsai-csp-utils/colab/install_rapids.py', 'r') as f:
    content = f.read()
    print("First 50 lines of install_rapids.py:")
    print('\n'.join(content.split('\n')[:50]))

First 50 lines of install_rapids.py:
#!/usr/bin/env python
import os, sys, io
import subprocess
from pathlib import Path

try:
  import pynvml
except:
  output = subprocess.Popen(["pip install pynvml"], shell=True, stderr=subprocess.STDOUT,
      stdout=subprocess.PIPE)
  for line in io.TextIOWrapper(output.stdout, encoding="utf-8"):
    if(line == ""):
      break
    else:
      print(line.rstrip())
  import pynvml
try:
  pynvml.nvmlInit()
except:
  raise Exception("""
                  Unfortunately you're in a Colab instance that doesn't have a GPU.

                  Please make sure you've configured Colab to request a GPU Instance Type.

                  Go to 'Runtime -> Change Runtime Type --> under the Hardware Accelerator, select GPU', then try again."""
  )
gpu_name = pynvml.nvmlDeviceGetName(pynvml.nvmlDeviceGetHandleByIndex(0))

# CFFI fix with pip
output = subprocess.Popen(["pip uninstall --yes cffi"], shell=True, stderr=subprocess.STDOUT,
    stdout=subprocess.PIPE)
fo

In [6]:
exec(open('rapidsai-csp-utils/colab/install_rapids.py').read(), globals())

Found existing installation: cffi 1.15.0
Uninstalling cffi-1.15.0:
  Successfully uninstalled cffi-1.15.0
  Using cached cffi-1.15.0-cp311-cp311-linux_x86_64.whl
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.18.0 requires cffi>=1.17.0, but you have cffi 1.15.0 which is incompatible.
rpy2 3.5.17 requires cffi>=1.15.1, but you have cffi 1.15.0 which is incompatible.
sed: can't read /usr/local/conda-meta/pinned: No such file or directory
Installing RAPIDS Stable 23.12
Starting the RAPIDS install on Colab.  This will take about 15 minutes.
/bin/sh: 1: conda: not found
/bin/sh: 1: mamba: not found
RAPIDS conda installation complete.  Updating Colab's libraries...


In [ ]:
import subprocess
import sys

# Install the missing RAPIDSAI packages with CUDA 12 support
missing_packages = [
    "cugraph-cu12",
    "cuspatial-cu12",
    "cuproj-cu12",
    "cuxfilter-cu12"
]

for package in missing_packages:
    print(f"Installing {package}...")
    try:
        subprocess.check_call([
            sys.executable, "-m", "pip", "install",
            package, "--no-cache-dir"
        ])
        print(f"✅ {package} installed successfully")
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to install {package}: {e}")
    print("-" * 50)

Installing cugraph-cu12...


In [ ]:
import subprocess
import sys

# Check for RAPIDSAI packages
rapidsai_packages = ['cudf', 'cuml', 'cugraph', 'cuspatial', 'cuproj', 'cuxfilter']

print("Checking RAPIDSAI package availability:")
for package in rapidsai_packages:
    try:
        __import__(package)
        print(f"✅ {package} - INSTALLED")
    except ImportError:
        print(f"❌ {package} - NOT INSTALLED")

# Also check pip list for any rapids packages
result = subprocess.run([sys.executable, "-m", "pip", "list"],
                       capture_output=True, text=True)
rapids_lines = [line for line in result.stdout.split('\n') if 'cu' in line.lower()]
print(f"\nInstalled packages with 'cu' in name:")
for line in rapids_lines:
    if line.strip():
        print(f"  {line}")

In [ ]:
# If pip installation fails, try conda
!conda install -c rapidsai -c conda-forge -c nvidia cudf cuml cugraph cuspatial cuproj -y

In [ ]:
# Install RAPIDS - Fixed version
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

import sys, os
import cudf  # GPU-accelerated pandas
import cuml  # GPU-accelerated scikit-learn
import cugraph  # GPU-accelerated network analysis

# Check what Python version and paths are available
print("Python version:", sys.version)
print("Available paths:")
for path in sys.path:
    print(f"  {path}")

# Find the actual dist-packages path dynamically
dist_packages_paths = [path for path in sys.path if 'dist-packages' in path]
site_packages_paths = [path for path in sys.path if 'site-packages' in path]

print(f"\nFound dist-packages paths: {dist_packages_paths}")
print(f"Found site-packages paths: {site_packages_paths}")

# If we found the paths, update sys.path
if dist_packages_paths:
    dist_package_index = sys.path.index(dist_packages_paths[0])
    # Construct the corresponding site-packages path
    site_packages_path = dist_packages_paths[0].replace('dist-packages', 'site-packages')
    sys.path = sys.path[:dist_package_index] + [site_packages_path] + sys.path[dist_package_index:]
    print(f"\nUpdated sys.path to include: {site_packages_path}")
else:
    print("\nNo dist-packages path found, skipping sys.path modification")

print("\nFinal sys.path:")
for i, path in enumerate(sys.path):
    print(f"  {i}: {path}")

# Execute the update modules script
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

That's it! Now we can run a variety of GPU acclerated graph mining algorithms.



## Part 1: Quantifing network vulnerability and robustness

While CPU calculations work well for sparse graphs, GPU acceleration significantly speeds-up analysis for dense graphs. To see this, lets run the code below that measures the robustness of a Barabási Albert (BA) graph at varying levels of density (i.e., number of edges per node).

In [ ]:
import time
from tqdm import tqdm

from graph_tiger.measures import run_measure
from graph_tiger.graphs import graph_loader

# controls graph density by varying the number of non-zeroes per row (i.e., number of edges per node in graph)
nnz_per_row = list(range(50, 501, 50))

cpu_times = []
gpu_times = []
for nnz in tqdm(nnz_per_row):
  graph = graph_loader(graph_type='BA', n=1000, m=nnz, seed=1)

  start_cpu = time.time()
  robustness_index = run_measure(graph, measure='average_vertex_betweenness', k=int(0.05 * len(graph)))
  end_cpu = time.time()

  start_gpu = time.time()
  robustness_index = run_measure(graph, measure='average_vertex_betweenness', k=12, use_gpu=True)  # ****** Replace with cuGraph version: https://docs.rapids.ai/api/cugraph/stable/api.html#module-cugraph.centrality.betweenness_centrality ******
  end_gpu = time.time()


  cpu_times.append(round(end_cpu - start_cpu, 2))
  gpu_times.append(round(end_gpu - start_gpu, 2))




Now lets plot the results (lower is better).

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(nnz_per_row, cpu_times, label='CPU')
plt.plot(nnz_per_row, gpu_times, label='GPU')
plt.xlabel('Number of edges per node (NNZ)')
plt.ylabel('Time (seconds)')
plt.title('Measuring Graph Robustness Runtime (CPU vs. GPU)')
plt.legend()
plt.show()

## Part 2. Simulating Cascading Failures in U.S. Electrical Grid
Cascading failures often arise as a result of natural failures or targeted attacks in a network. There are 3 main processes governing the network simulation:

- the **capacity** of each node (<img src="https://render.githubusercontent.com/render/math?math=c_v">) in the network, e.g., power substation capacity.

- the **load** of each node (<img src="https://render.githubusercontent.com/render/math?math=l_v">) in the network, e.g., power substation load level

- network **redundancy** (*r*) representing the amount of reserve capacity present in the network i.e., auxiliary support systems.

When a node is attacked it becomes "overloaded", causing it to fail and requiring the load be distributed to its neighbors. When defending a node, we increase it’s capacity to protect against attacks. With just these 3 parameters, we can setup a cascading failure simulation. Below, we show how to load a graph representing the U.S. electrical grid and setup the simulation parameters.

In [ ]:
from graph_tiger.graphs import graph_loader

graph = graph_loader('electrical')

params = {
   'runs': 1,  # number of times to run the simulation
   'steps': 100,  # number of time steps to run each simulation
   'seed': 1,  # for repoducibility

   'l': 0.8,  # network load [0, 1]
   'r': 0.2,  # network redundancey [0, 1]
   'c': int(0.1 * len(graph)),  # load capacity approximation

   'robust_measure': 'largest_connected_component',  # measure of network health
}

### Setting up a Targeted Attack
To run the attack we just have to modify a few simulation parameters. We set the attack to remove 30 nodes in the graph (e.g., power grid substations) with highest degree centrality "id_node". As you can imagine, there are many different strategies that can be used to attack the grid, however, by selecting degree centrality we can find "central" nodes in the network with many power lines (edges) connected to the substations (nodes).

In [ ]:
params.update({
   'k_a': 30,  # number of nodes to attack
   'attack': 'id_node',  # attack strategy
})

Now lets run the simulation and plot the results!

In [ ]:
from graph_tiger.cascading import Cascading

cascading = Cascading(graph, **params)
results = cascading.run_simulation()

cascading.plot_results(results)

### 1. Collapsing a network

Now try modifying the code to find the minimal attack necessary to collapse the electrical grid. To do this, plot the "health" of the network as measured by the robustness measure (i.e., "largest_connected_component") at the end of each simulation, against the attack strength ("k_a").

**Hint:** electrical networks are fragile to targeted attacks, try removing just a few nodes.

In [ ]:
sim_results =[]

params['attack'] = 'id_node'
attack_strengths = list(range(0, 6, 1))

for k_a in tqdm(attack_strengths):
  params['k_a'] = k_a

  cascading = Cascading(graph, **params)
  results = cascading.run_simulation()

  lcc_at_end = results[-1]
  sim_results.append(lcc_at_end)

%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(attack_strengths, sim_results)
plt.xlabel('Attack strength')
plt.ylabel('Largest connected component')
plt.title('Effect of Attack Strength in Cascading Failures')

# Part 3. Simulating Computer Virus Spread Across Router Network

The flu-like susceptible-infected-susceptible (SIS) is a popular model that allows us to simulate the spread of viruses across networks (graphs). Each node in the SIS model can be in one of two states, infected *I* or susceptible *S*, and at each time step *t*, an infected node has a probability *β* of infecting each of it's uninfected neighbors. After this, each infected node has a probability *δ* of healing and becoming susceptible again.

It’s been shown there's a direct correlation between the graph's topology as measured through the spectral radius (largest eigenvalue) of the graph, and the virus remaining endemic. The exact relationship between a virus's strength (*s*), birth rate (*β*), death rate (*δ*) and spectral radius (*λ1*) is s=λ1⋅b/d, where a larger *s* means a stronger virus. With just these 3 parameters, we can setup a computer virus simulation.

Below, we (1) load the Autonomous systems AS-733 network, which is a graph of routers comprising the internet; and (2) setup the simulation parameters.  

In [ ]:
from graph_tiger.graphs import graph_loader

graph = graph_loader('as_733')

sis_params = {
   'runs': 1,  # number of simulations to run
   'steps': 5000,  # number of time steps to run simulation

   'model': 'SIS',
   'b': 0.00208,  # virus birth rate
   'd': 0.01,  # virus death rate
   'c': 0.3,  # fraction of the network that starts infected
}

Now lets run the simulation and plot the results! In the figure below, we see that without intervention the virus remains endemic on the router network.

In [ ]:
from graph_tiger.diffusion import Diffusion

diffusion = Diffusion(graph, **sis_params)
results = diffusion.run_simulation()

diffusion.plot_results(results)

While we do not have control over the virus strength (*s*), we can maniuplate the underlying toplogy of the router network to make it harder for the virus to spread. The question is, how do we optimally modify the network to reduce the spread of the virus? While a naive solution may be to disconnect the whole network, this isn't very practical since everyone would loose internet access! Instead, we need a strategy that carefully vaccinates a few nodes (routers) against the virus.

### 2. Optimally Vaccinating a Network

Now lets compare the efficacy of 4 vaccination strategies when vaccinating only 3 nodes in the network:

1. [netshield](https://graph-tiger.readthedocs.io/en/latest/defenses.html#graph_tiger.defenses.get_node_ns) ('ns_node')
2. [id_node](https://graph-tiger.readthedocs.io/en/latest/defenses.html#graph_tiger.defenses.get_node_id) ('id_node')
2. [rd_node](https://graph-tiger.readthedocs.io/en/latest/defenses.html#graph_tiger.defenses.get_node_rd) ('rd_node')
3. [ib_node](https://graph-tiger.readthedocs.io/en/latest/defenses.html#graph_tiger.defenses.get_node_ib) ('ib_node')

To implement a defense strategy you just have to modify a few simulation parameters.

In [ ]:
sis_params.update({
    'diffusion': 'min',  # we want to minimize the ability of the virus to propagate,
    'method': 'ns_node',  # use the Netshield technique
    'k': 15  # vaccinate 5 nodes according the selected strategy
})

Does each strategy manage to contain the virus (i.e., less than 1\% infected population)? Which strategy has the lowest infected population at the end of the simulation? Setup and run each simulation and compare the results to the unvaccinated network.

In [ ]:
methods = ['ns_node', 'id_node', 'rd_node', 'pr_node']

for method in methods:
  sis_params['method'] = method

  diffusion = Diffusion(graph, **sis_params)
  results = diffusion.run_simulation()

  print('Percent of network that remains infected at end of simulation using {} vaccination technique is {}%'.format(method, round((results[-1] / len(graph)) * 100, 2)))
  diffusion.plot_results(results)